# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, concat, desc, asc, explode, lit, min, max, split, udf
from pyspark.sql.functions import to_timestamp, year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql.types import IntegerType, StringType

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA, RegexTokenizer, StandardScaler, StopWordsRemover, StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import re
import datetime

In [2]:
# create a Spark session

spark = SparkSession.builder \
    .appName("Sparkify") \
    .getOrCreate()

In [3]:
spark

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [4]:
df = spark.read.json('../data/mini_sparkify_event_data.json')

In [5]:
df.head(2)

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30'),
 Row(artist='Five Iron Frenzy', auth='Logged In', firstName='Micah', gender='M', itemInSession=79, lastName='Long', length=236.09424, level='free', location='Boston-Cambridge-Newton, MA-NH', method='PUT', page='NextSong', registration=1538331630000, sessionId=8, song='Canada', status=200, ts=1538352180000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"', userId='9')]

In [6]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [7]:
df.describe().show(1,vertical=True)

-RECORD 0---------------
 summary       | count  
 artist        | 228108 
 auth          | 286500 
 firstName     | 278154 
 gender        | 278154 
 itemInSession | 286500 
 lastName      | 278154 
 length        | 228108 
 level         | 286500 
 location      | 278154 
 method        | 286500 
 page          | 286500 
 registration  | 278154 
 sessionId     | 286500 
 song          | 228108 
 status        | 286500 
 ts            | 286500 
 userAgent     | 278154 
 userId        | 286500 
only showing top 1 row



In [8]:
df.distinct().count()

286500

In [11]:
df=df.filter(df.userId != '')

In [12]:
df=df.withColumn('event_timestamp', to_timestamp(df['ts']/1000))
df=df.withColumn('registration_timestamp', to_timestamp(df['registration']/1000))

In [17]:
df.createOrReplaceTempView("events")
user_table = spark.sql("""
                        SELECT DISTINCT
                            userId as user_id, 
                            firstName as first_name, 
                            lastName as last_name, 
                            gender, 
                            location,
                            registration,
                            registration_timestamp
                        FROM events
                        """)
user_table.show(3)

+-------+----------+---------+------+--------------------+-------------+----------------------+
|user_id|first_name|last_name|gender|            location| registration|registration_timestamp|
+-------+----------+---------+------+--------------------+-------------+----------------------+
|    143|     Molly| Harrison|     F|Virginia Beach-No...|1534255113000|   2018-08-14 15:58:33|
|    114|     Titus|    Jones|     M|Las Vegas-Henders...|1536831228000|   2018-09-13 11:33:48|
|     30|     Colin|  Freeman|     M|     Bakersfield, CA|1538173362000|   2018-09-29 00:22:42|
+-------+----------+---------+------+--------------------+-------------+----------------------+
only showing top 3 rows



In [18]:
user_table.groupBy('user_id').count().orderBy('count', ascending=False).show()

+-------+-----+
|user_id|count|
+-------+-----+
| 100010|    1|
| 200002|    1|
|    125|    1|
|    124|    1|
|      7|    1|
|     51|    1|
|     15|    1|
|     54|    1|
|    155|    1|
|    132|    1|
|    154|    1|
| 100014|    1|
|     11|    1|
|    101|    1|
|    138|    1|
| 300017|    1|
|     29|    1|
|     69|    1|
| 100021|    1|
|     42|    1|
+-------+-----+
only showing top 20 rows



In [19]:
user_table.filter(user_table['user_id']==70).show()

+-------+----------+---------+------+-----------+-------------+----------------------+
|user_id|first_name|last_name|gender|   location| registration|registration_timestamp|
+-------+----------+---------+------+-----------+-------------+----------------------+
|     70|    Chayse|   Keller|     M|Jackson, MS|1529643103000|   2018-06-22 06:51:43|
+-------+----------+---------+------+-----------+-------------+----------------------+



In [20]:
df.createOrReplaceTempView("events")
event_table = spark.sql("""
                    SELECT DISTINCT
                        userId as user_id,
                        sessionId as session_id,
                        itemInSession as item_in_session,
                        page as event,
                        auth,
                        level,

                        userAgent as user_agent,
                        ts as event_timestamp_unix,
                        event_timestamp                   
                FROM events
                ORDER BY event_timestamp_unix DESC
                """) 


In [21]:
event_table.filter(event_table['user_id']==70).toPandas().head(20)

C:\Users\thoma\anaconda3\Lib\site-packages\pyspark\sql\pandas\conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,user_id,session_id,item_in_session,event,auth,level,user_agent,event_timestamp_unix,event_timestamp
0,70,1940,91,Cancellation Confirmation,Cancelled,paid,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",1542234867000,2018-11-14 23:34:27
1,70,1940,90,Cancel,Logged In,paid,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",1542234843000,2018-11-14 23:34:03
2,70,1940,89,Downgrade,Logged In,paid,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",1542234842000,2018-11-14 23:34:02
3,70,1940,88,NextSong,Logged In,paid,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",1542234810000,2018-11-14 23:33:30
4,70,1940,87,Thumbs Up,Logged In,paid,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",1542234742000,2018-11-14 23:32:22
5,70,1940,86,NextSong,Logged In,paid,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",1542234741000,2018-11-14 23:32:21
6,70,1940,84,NextSong,Logged In,paid,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",1542234451000,2018-11-14 23:27:31
7,70,1940,85,Downgrade,Logged In,paid,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",1542234451000,2018-11-14 23:27:31
8,70,1940,83,NextSong,Logged In,paid,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",1542234208000,2018-11-14 23:23:28
9,70,1940,82,NextSong,Logged In,paid,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",1542233958000,2018-11-14 23:19:18


# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.

In [ ]:
spark.stop()